1. Є такі дані про транзакції в період з 01.12.2010 по 12.09.2011:
InvoiceNo – номер транзакції
StockCode – код товару
Description – опис товару
Quantity – кількість одиниць товару, доданих на замовлення
InvoiceDate – дата транзакції
UnitPrice – ціна за одиницю товару
CustomerID – id клієнта
Country – країна, де мешкає клієнт
Імпортуйте дані data.csv.zip (https://drive.google.com/file/d/1-yIXpdviyLVGqfBtgrLrw033ZtImVs0f/view?usp=sharing) з кодуванням ISO-8859-1. 
Запишіть отриманий датафрейм у retail, а назви колонок збережіть змінну retail_columns.

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
data_url = 'https://drive.google.com/file/d/1-yIXpdviyLVGqfBtgrLrw033ZtImVs0f/view?usp=sharing'
data_path = 'https://drive.google.com/uc?id=' + data_url.split('/')[-2]
retail = pd.read_csv(data_path, encoding='ISO-8859-1')

In [3]:
retail.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom


In [4]:
retail_columns = retail.columns

In [5]:
retail_columns

Index(['InvoiceNo', 'StockCode', 'Description', 'Quantity', 'InvoiceDate',
       'UnitPrice', 'CustomerID', 'Country'],
      dtype='object')

2. Перевірте, чи зустрічаються в даних спостереження що повторюються, і в якості відповіді вкажіть їх кількість. Якщо вони є, то видаліть їх із retail.

In [6]:
retail.shape

(541909, 8)

In [7]:
retail.drop_duplicates(inplace=True)

In [8]:
retail.shape

(536641, 8)

3. ⭐️Завдання зі зірочкою!⭐️
Дані містять у собі записи як і про успішні транзакції, так і про скасовані. Якщо користувач скасував замовлення, на початку номера транзакції (InvoiceNo) ставиться C (canceled).
Скільки всього замовлень скасували користувачі?

In [9]:
retail.InvoiceNo.str.startswith('C').sum()

9251

In [10]:
retail.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom


4. Тепер відфільтруйте дані і залиште в retail тільки ті замовлення, де Quantity > 0. Як відповідь вкажіть кількість рядків, що залишилися.

In [11]:
retail = retail.query('Quantity > 0')

In [12]:
retail.shape

(526054, 8)

5. 💀 Складне завдання! 💀
Порахуйте кількість замовлень для кожного користувача (CustomerID) із Німеччини (Germany). Залишіть лише тих, хто здійснив більше N транзакцій (InvoiceNo), де N – 80% процентиль. Запишіть отримані id користувачів у germany_top (не весь датафрейм, тільки id). Ідентифікатор замовлення – InvoiceNo. Для кожного замовлення даних може зустрічатися більше 1 рядка.

In [13]:
german_buyers = retail.query('Country == "Germany"') \
                    .groupby('CustomerID', as_index=False) \
                    .agg({'InvoiceNo': pd.Series.nunique}) \
                    .rename(columns={'InvoiceNo': 'NumberOrders'})


In [14]:
percentil_80 = german_buyers.NumberOrders.quantile(0.8)

In [15]:
germany_top = german_buyers.query('NumberOrders > @percentil_80').CustomerID

6. ⭐️Завдання зі зірочкою!⭐️
Використовуючи об'єкт з id користувачів (germany_top), отриманий на попередньому етапі, відфільтруйте спостереження і залиште в даних записи тільки по користувачам, що нас цікавлять. Результуючий датафрейм запишіть top_retail_germany.

In [16]:
top_retail_germany = retail.query('CustomerID in @germany_top')

7. Згрупуйте top_retail_germany за кодом товару (StockCode). Який із продуктів додавали до кошика найчастіше, крім POST?
Note: одним замовленням вважається одноразове придбання будь-якої кількості товару, тобто. без урахування Quantity.

In [17]:
top_retail_germany.query('StockCode != "POST"').StockCode.value_counts().head(1)

22326    52
Name: StockCode, dtype: int64

In [18]:
top_retail_germany.query('StockCode !="POST"') \
                .groupby('StockCode', as_index=False) \
                .agg({'Description': 'count'}) \
                .sort_values('Description', ascending=False) \
                .head(1)

,StockCode,Description
409,22326,52


8. Повернемося до аналізу повного датасету retail. Створіть колонку Revenue із сумою покупки, використовуючи колонки Quantity та UnitPrice.

In [19]:
retail['Revenue'] = retail.Quantity * retail.UnitPrice

In [20]:
retail.drop(columns='Revenue', inplace=True)

In [21]:
retail = retail.assign(
            Revenue = retail.Quantity * retail.UnitPrice,
            Revenue1 = retail.Quantity + retail.UnitPrice
        )

In [22]:
retail.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Revenue,Revenue1
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom,15.30,8.55
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom,20.34,9.39
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom,22.00,10.75
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom,20.34,9.39
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom,20.34,9.39


9. Для кожної транзакції (InvoiceNo) порахуйте фінальну суму замовлення. Як відповідь вкажіть топ-5 (InvoiceNo) за сумою замовлення (через кому з пробілом і в порядку зменшення TotalRevenue)

Наприклад, для наступного прикладу...

InvoiceNo StockCode Quantity InvoiceDate UnitPrice CustomerID Revenue
536365 85123A 6 2010-12-01 08:26:00 2.55 17850.0 15.30
536365 71053 6 2010-12-01 08:26:00 3.39 17850.0 20.34
555555 71053 1 2010-12-02 18:00:00 3.39 17850.0 3.39
...сума замовлення дорівнюватиме:

InvoiceNo TotalRevenue
536365 35.64
555555 3.39

І відповіддю буде:
536365, 555555

In [23]:
sorted_revenues = retail \
    .groupby('InvoiceNo', as_index=False) \
    .agg({'Revenue': 'sum'}) \
    .rename(columns={'Revenue': 'TotalRevenue'}) \
    .sort_values('TotalRevenue', ascending=False)

In [24]:
sorted_revenues

,InvoiceNo,TotalRevenue
20689,581483,168469.60
2202,541431,77183.60
17582,574941,52940.94
18251,576365,50653.91
9034,556444,38970.00
...,...,...
10074,558724,0.00
10075,558725,0.00
10150,558889,0.00
20726,A563186,-11062.06


In [25]:
print(sorted_revenues.InvoiceNo[:5].str.cat(sep=', '))

581483, 541431, 574941, 576365, 556444
